<a href="https://colab.research.google.com/github/tcglarry/kg_power_line/blob/master/vsb_power_line_eda_correct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# Step 1
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


E: Package 'python-software-properties' has no installation candidate


In [3]:

# Step 2
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install pyarrow

In [0]:
!sudo chmod 444 'drive/My Drive/kaggle/vsb_power_line/data/train.parquet'

In [0]:
!cp 'drive/My Drive/kaggle/vsb_power_line/data/train.parquet' .

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow.parquet as pq
import matplotlib as mpl
mpl.style.use('default')
from PIL import Image
import seaborn as sns
%matplotlib inline

In [0]:
src= 'drive/My Drive/kaggle/vsb_power_line/data/'
src_train_idx_1 = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/train/1/'
src_train_idx_0 = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/train/0/'
src_valid_idx_1 = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/validation/1/'
src_valid_idx_0 = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/validation/0/'

In [0]:
#!rm 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/validation/1/'*

In [11]:
import os
os.listdir('drive/My Drive/kaggle/vsb_power_line/data/train_test_data/train/')

['0', '1']

In [12]:
train_df = pd.read_csv(src+'metadata_train.csv')
print (train_df.shape)
train_df['target'].value_counts()

(8712, 4)


0    8187
1     525
Name: target, dtype: int64

In [13]:
idx_1 = train_df.loc[train_df['target']==1,'signal_id'].tolist()
idx_0 = train_df.loc[train_df['target']==0,'signal_id'].tolist()

print (len(idx_1)+len(idx_0))

8712


In [14]:
test_ratio = 0.2
valid_size_1 = int(525*test_ratio)
valid_size_0 = int(8187 * test_ratio)

print (valid_size_1, valid_size_0)

105 1637


In [15]:
valid_idx_1 = np.random.choice(idx_1, size= valid_size_1,replace=False)
train_idx_1 = [i for i in idx_1 if i not in valid_idx_1]

valid_idx_0 = np.random.choice(idx_0, size= valid_size_0,replace=False)
train_idx_0 = [i for i in idx_0 if i not in valid_idx_0]

print (len(valid_idx_1), len(train_idx_1) , len(valid_idx_0), len(train_idx_0))

105 420 1637 6550


In [0]:
def save_file_to(i):
  if i in train_idx_1:
    save_to = src_train_idx_1
  elif i in train_idx_0:
    save_to = src_train_idx_0
  
  elif i in valid_idx_1:
    save_to = src_valid_idx_1

  elif i in valid_idx_0:
    save_to = src_valid_idx_0
    
  else:
    print (f'i = {i},   something wrong')
    
  return save_to 

In [0]:



def sample(signal, kernel_size):
  sampled = np.zeros((signal.shape[0],  signal.shape[1]//kernel_size))
  print ('sampledshape',sampled.shape)
  for i in range(signal.shape[1]//kernel_size):
    begin = kernel_size * i
    #print (f'begin = {begin}')
    end = min(kernel_size * (i + 1), signal.shape[1])
    #print (f'end = {end}')
    sampled[:,  i] = np.mean(signal[:,  begin:end], axis=1)
    #print (f'from begin {begin } to end {end} mean value {sampled[:, :, i] } selected ')
  return sampled


In [0]:

def signals_to_img(fname,window=100):

  #plt.imshow(FNAME)
  #plt.show()
  img = Image.open(fname).convert('RGB')
  #print (img.size)
  #img = img.resize((750,500))
  x = np.array(img)
  #print (x.shape)

  img.save(fname)


In [0]:

def make_fig(j,sampled):

  length = sampled.shape[0]


  for i in range(length):
    
    idx = i+j*484
    save_to = save_file_to(idx)


    fig = plt.figure(figsize=(8, 4))
    


    plt.xticks([])
    plt.yticks([])
    
    sns.despine(bottom=True, left=True)
    plt.plot(sampled[i, :],color='r')
    #plt.plot(sampled[i*11, :], label='Phase 1')
    #plt.plot(sampled[i*11, :], label='Phase 2')
    #plt.legend()
    #print ('j=', j, 'i=',i, 'save_to =', save_to)
    fname = save_to+'signal_img_'+str(i+j*484)+'.png'
    plt.savefig(fname, format='png')
    plt.close(fig)
    signals_to_img(fname)
    fig.clf()

    #plt.despine(left=True,bottom=True)
    #plt.show()

In [50]:

#train_df = pq.read_pandas('train.parquet').to_pandas()

for j in range(10,18,1):
  
  subset_train = pq.read_pandas('train.parquet', columns=[str(i+j*484) for i in range(484)]).to_pandas()
  subset_value = np.array(subset_train.values).T
  print (subset_value.shape)
  sampled = sample(subset_value,100)
 
  make_fig(j,sampled)
  if j % 3 == 0:
    
    print (f'batch {j} /18 ok ')
  
print ('total complted')

(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
batch 12 /18 ok 
(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
batch 15 /18 ok 
(484, 800000)
sampledshape (484, 8000)
(484, 800000)
sampledshape (484, 8000)
total complted


In [53]:
import os
len( os.listdir(src_train_idx_1)) + len( os.listdir(src_valid_idx_1))


525

In [54]:
len( os.listdir(src_train_idx_0)) + len( os.listdir(src_valid_idx_0))


8187